<a href="https://colab.research.google.com/github/xandruUu/super-store-data-Oesia/blob/main/notebooks/superStoreDataOesia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/xandruUu/super-store-data-Oesia.git

Cloning into 'super-store-data-Oesia'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 36 (delta 9), reused 28 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (36/36), 479.30 KiB | 7.61 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [2]:
%cd /content/super-store-data-Oesia

/content/super-store-data-Oesia


In [18]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt

# Configuración de la ruta raíz
if os.path.exists('/content'):
    repo_path = '/content/super-store-data-Oesia'
else:
    actual_path = os.getcwd()
    if 'notebooks' in actual_path:
        repo_path = os.path.dirname(actual_path)
    else:
        repo_path = actual_path

# Definición de rutas de salida
folder_path = os.path.join(repo_path, 'data', 'processed')
file_path = os.path.join(folder_path, 'train_cleaned.csv')

# Nos aseguramos que la carpeta existe desde el principio
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Añado la ruta al sistema e importo
if repo_path not in sys.path:
    sys.path.append(repo_path)

from src.data_superstore import solucion_super_store_data

#Super-Store-Data Analisis

Con este notebook lo que consigo es analizar el dataset elegido y resolver los problemas
detectados en el dataset *Superstore Sales Dataset*.

Mi objetivo es:
- Identificar los errores en los datos.
- Conseguir justificar por qué son un problema para el dataset.
- Aplicar una solución reproducible con Python.
- Verificar visualmente el resultado.

In [4]:
df = pd.read_csv('data/raw/train.csv')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int64  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

## PROBLEMA 1 IDENTIFICADO: Códigos postales Nulos

He detectado registros en el campo **postal_code** vacíos

Esto puede llegar a suponer un problema para los analísis geografícos y logísticos generando un problema de negocio que resulta en una una perdida de dinero.

In [6]:
codigosPostales_nulos = df.loc[df['Postal Code'].isnull(), ['City', 'State', 'Postal Code']]
codigosPostales_nulos.head()

,City,State,Postal Code
2234,Burlington,Vermont,NaN
5274,Burlington,Vermont,NaN
8798,Burlington,Vermont,NaN
9146,Burlington,Vermont,NaN
9147,Burlington,Vermont,NaN


### Análisis

Todos los registros con el código postal nulo pertenecen a:
- Ciudad: Burlington
- Estado: Vermont

Usamos ciudad y estado, ya que si dependiéramos solo de la ciudad, nos podríamos confundir porque existen otras ciudades llamadas Burlington en otros estados.

## Problema 2: Códigos Postales sin Ceros Iniciales

Los códigos postales en EE. UU. deben tener exactamente **5 dígitos**.

Sin embargo, al almacenarse como números, los ceros iniciales
se pierden (ej. `07090` → `7090`).

O los .0 se añaden como un 0 y dan 7 digitos como codigo postal.

O Los que empiezan por 0 y 00 se pierden y se quedan codigos postales de 3 y 4 digitos.

Esto es especialmente común en la región noreste de EE. UU.
y genera errores en análisis logísticos y geográficos.

In [7]:
df['Postal Code'].astype(str).str.len().value_counts()

,count
Postal Code,
7,9360
6,429
3,11


## Problema 3: Inconsistencias en Nombres de Producto

Se detecta que un mismo `Product ID` está asociado a múltiples nombres, lo que puede provocar errores en estadísticas y en el conteo total de dicho producto.

In [9]:
product_counts = df.groupby('Product ID')['Product Name'].nunique()
inconsistent_ids = product_counts[product_counts > 1].index

df_errors = df[df['Product ID'].isin(inconsistent_ids)][['Product ID', 'Product Name']].copy()
df_errors = df_errors.drop_duplicates().sort_values(by='Product ID')

pd.set_option('display.max_rows', None)
print(f"Se han encontrado {len(inconsistent_ids)} IDs con nombres conflictivos:\n")
display(df_errors)

Se han encontrado 32 IDs con nombres conflictivos:



,Product ID,Product Name
2471,FUR-BO-10002213,"Sauder Forest Hills Library, Woodland Oak Finish"
2115,FUR-BO-10002213,DMI Eclipse Executive Suite Bookcases
66,FUR-CH-10001146,"Global Value Mid-Back Manager's Chair, Gray"
128,FUR-CH-10001146,"Global Task Chair, Black"
1459,FUR-FU-10001473,DAX Wood Document Frame
2204,FUR-FU-10001473,"Eldon Executive Woodline II Desk Accessories, ..."
3831,FUR-FU-10004017,"Executive Impressions 13"" Chairman Wall Clock"
234,FUR-FU-10004017,Tenex Contemporary Contur Chairmats for Low an...
1385,FUR-FU-10004091,"Eldon 200 Class Desk Accessories, Black"
293,FUR-FU-10004091,"Howard Miller 13"" Diameter Goldtone Round Wall..."


### Solución Implementada: Normalización de Catálogo

Para resolver las inconsistencias en los datos de origen, he aplicado la siguiente lógica de procesamiento:

* **Priorización del ID como Fuente de Verdad:** El sistema confía exclusivamente en el `Product ID`. Ignoro variaciones en las cadenas de texto y asigno automáticamente el primer nombre asociado que se encuentra en el registro.

* **Estandarización de Códigos Postales (ZIP Codes):** * Se ha implementado una lógica de **Padding a 5 dígitos**.

    * Esto recupera los ceros iniciales (`0` o `00`) esenciales para los estados del Noreste (como Vermont, Massachusetts o Connecticut), evitando que se traten como números recortados y cumpliendo con el estándar de la USPS.

* **Normalización Estructural del CSV:** He reestructurado el archivo para garantizar una relación **1:1** entre ID y Nombre, unificando el dataset.

* **Optimización del Análisis de Ventas:** Al eliminar la mezcla  de nombres, evitamos la duplicación de registros y aseguramos un analísis de ventas 100% limpio.

In [10]:
df_clean = solucion_super_store_data(df.copy())
df_clean.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [11]:
df_clean['Postal Code'].isnull().sum()
df_clean.groupby('Product ID')['Product Name'].nunique().max()

1

In [19]:
df_clean.to_csv(file_path, index=False)